#**Import Lbraries**

In [1]:
!pip install gtts

import json
from gtts import gTTS
from IPython.display import Audio, display
import os


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\EMAM ABD EL MONSEF\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


#**Choose files**

In [2]:
# اختيار الملفات
files = ["stories_ar.json", "stories_en.json"]

# إنشاء مجلد لحفظ الملفات الصوتية إذا لم يكن موجوداً
if not os.path.exists("audio_stories"):
    os.makedirs("audio_stories")

def detect_language_from_filename(filename):
    if "_ar" in filename:
        return "ar"
    elif "_en" in filename:
        return "en"
    else:
        return "ar"


#**Download stories**

In [3]:
all_stories = []
for file in files:
    lang = detect_language_from_filename(file)
    with open(file, "r", encoding="utf-8") as f:
        stories_data = json.load(f)
        for story in stories_data:
            story["lang"] = lang
            all_stories.append(story)


#**Sound creation function**

In [4]:
def create_audio_story(story):
    lang = story["lang"]

    text = f"""
    {story.get('category', '')}
    {story.get('title', '')}
    {story.get('content', '')}
    """

    if "lessons" in story and story["lessons"]:
        lessons_text = ", ".join(story["lessons"])
        if lang == "ar":
            text += f"\nالدروس المستفادة: {lessons_text}"
        else:
            text += f"\nLessons: {lessons_text}"

    filename = f"audio_stories/story_{story['id']}_{story['title'].replace(' ', '_')}_{lang}.mp3"
    tts = gTTS(text=text.strip(), lang=lang, slow=False)
    tts.save(filename)
    return filename, text.strip()

#**search_stories**

In [10]:

def search_stories(keyword):
    results = []
    for story in all_stories:
        if (
            keyword.lower() in story.get("title", "").lower()
            or keyword.lower() in story.get("content", "").lower()
            or keyword.lower() in story.get("category", "").lower()
        ):
            results.append(story)
    return results



#**Use**

In [12]:
keyword = input("اكتب كلمة من عنوان القصة: ")
results = search_stories(keyword)


if results:
    print("\n📚 القصص المتاحة:")
    for i, story in enumerate(results, 1):
        print(f"{i}. {story['title']} ({story['lang']})")

    choice = int(input("\nاختر رقم القصة: ")) - 1
    if 0 <= choice < len(results):
        selected_story = results[choice]
        print(f"\n📖 القصة المختارة: {selected_story['title']}")
        audio_file, text = create_audio_story(selected_story)

        print("\n--- نص القصة ---")
        print(text)
        print("\n🎤 تشغيل القصة صوتيًا...")
        display(Audio(audio_file, autoplay=True))

    else:
        print("❌ اختيار غير صالح")
else:
    print("⚠️ لم يتم العثور على أي قصة بهذا العنوان")

⚠️ لم يتم العثور على أي قصة بهذا العنوان
